### 0. RNN(Recurrent Neural Network)
cf) https://wikidocs.net/22886<br/>
    https://www.edwith.org/deeplearningchoi/lecture/15840/
#### 1) RNN
- RNN은 시퀀스 모델(입력과 출력을 시퀀스 단위로 처리하는 모델)<br/>
cf) 재귀 신경망(Recursive NN)과는 전혀 다른 개념
<img src="img/3_1.png" width="300" height="300">


- 메모리 셀(초록색 셀)은 이전의 값을 기억하려고 하는 __메모리 역할__
- 메모리 셀이 출력층 방향으로 또는 다음 시점 t+1의 자신에게 보내는 값(재귀적)을<br/> __은닉상태(hidden state)__라고 함


- 수식
<img src="img/3_2.png" width="400" height="300">
<img src="img/3_3.png" width="500" height="300">

In [1]:
from tensorflow import keras

In [3]:
from tensorflow.keras.layers import SimpleRNN

In [ ]:
# RNN 층을 추가하는 코드

model.add(SimpleRNN(hidden_size, input_shape=(timesteps, input_dim)))
model.add(SimpleRNN(hidden_size, input_length=M, input_dim=N)) # 같은 코드

- hidden_size($D_h$) : 은닉 상태의 크기(output_dim과도 동일). 중소형 모델의 경우, 보통 128, 256, 512, 1024 등의 값을 가짐
- timesteps($d$) : 입력 시퀀스의 길이(input_length라고도 함)
- input_dim : 입력의 크기(dimensionality of word representation)
- 위 코드는 은닉층만을 위한 코드임(출력층 x)
<img src="img/3_4.png" width="100" height="300">

#### 2) 출력의 형태
- return_sequences 옵션<br/>
    ○ 출력값이 (batch_size, output_dim) : 최종 시점의 은닉 상태만을 리턴(왼쪽 그림)<br/>
    ○ 출력값이 (batch_size, timesteps, output_dim) : 은닉 상태들의 전체 시퀀스 리턴(오른쪽 그림)
    cf) batch_size는 hidden state의 개수
<img src="img/3_5.png" width="300" height="300">

In [8]:
from tensorflow.python.keras.models import Sequential

In [10]:
# 출력값이 (batch_size=2, output_dim=10) 

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [12]:
# 출력값이 (batch_size=8, timesteps=2, output_dim=10)

model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Deep RNN

model = Sequential()
model.add(SimpleRNN(hidden_size, return_sequences = True))
model.add(SimpleRNN(hidden_size, return_sequences = True))

#### 3) 양방향 RNN (Birectional RNN)
- 양방향 RNN은 기본적으로 두 개의 메모리 셀을 사용  
: __앞 시점의 은닉상태(Forward States), 뒤 시점의 은닉상태(Backward States)__
<img src="img/3_6.png" width="300" height="300">

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Bidirectional

In [ ]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))

In [ ]:
# 은닉층 하나 더 추가

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))

### 1. 긴 시퀀스를 기억할 수 있는 LSTM (Long Short-Term Memory units) 레이어
- __장기 의존성 문제(the problem of Long-Term Dependencies)__  
: SimpleRNN(Vanilla RNN)은 시점(time step)이 길어질수록 앞의 정보가 뒤로 충분히 전달되지 못하는 문제
<img src="img/3_7.png" width="450" height="300"><br/>
- LSTM은 은닉층의 메모리 셀에 __입력 게이트, 망각 게이트, 출력 게이트__를 추가하여 불필요한 기억을 지우고, 기억해야할 것들을 정함
- __Cell State__($C_t$) : t시점의 셀 상태


- $W_{xi},W_{xg},W_{xf},W_{xo}$ : $x_t$와 함께 각 게이트에서 사용되는 4개의 가중치
- $W_{hi},W_{hg},W_{hf},W_{ho}$ : $h_{t−1}$와 함께 각 게이트에서 사용되는 4개의 가중치
- $b_i,b_g,b_f,b_o$ : 각 게이트에서 사용되는 4개의 편향

#### 1) 입력 게이트
- 현재 정보를 기억하기 위한 게이트
-     $i_t=\sigma(W_{xi}x_t+W_{hi}h_{t-1}+b_i)$<br/>
- $g_t=tanh(W_{xg}x_t+W_{hg}h_{t-1}+b_g)$
<img src="img/3_8.png" width="300">

    
    
- $i_t, g_t$ : 현재 시점의 $x_t$와 이전 시점의 은닉 상태 $h_t$의 가중합을 각각 시그모이드 함수와 하이퍼볼릭탄젠트 함수를 지나 출력되는 값 
- $i_t$ : 시그모이드 함수를 지나 0~1사이의 값 출력  
=> $g_t$를 얼마나(%) 반영할 지

#### 2) 삭제 게이트
- 기억을 삭제하기 위한 게이트  
- $f_t=\sigma(W_{xf}x_t+W_{hf}h_{t-1}+b_f)$
<img src="img/3_9.png" width="300">
    
    
    
- $f_t$ : 현재 시점의 $x_t$와 이전 시점의 은닉 상태 $h_t$의 가중합을 시그모이드 함수를 지나 출력되는 값 (0~1 => 얼마나 반영할 지)  
=> 삭제 과정을 거친 정보의 양 (0에 가까울수록 정보 많이 삭제)

#### 3) 셀 상태(장기 상태)
- $C_t=f_t∘C_{t−1}+i_t∘g_t$  (∘는 원소별 곱을 의미)
<img src="img/3_10.png" width="300">

- 삭제 게이트는 이전 시점의 입력을 얼마나 반영할지를 의미하고, 입력 게이트는 현재 시점의 입력을 얼마나 반영할지를 결정함

#### 4) 출력 게이트와 은닉 상태(단기 상태)
- $o_t=\sigma(W_{xo}x_t+W_{ho}h_{t-1}+b_o)$ 
- $h_t=o_t∘tanh(c_t)$
<img src="img/3_11.png" width="300">
- $o_t$ : 현재 시점 t의 x값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지난 값<br/> 
    => 현재 시점 t의 은닉 상태(단기 상태, $h_t$)를 얼마나 빼낼 지 결정
- 장기 상태($c_t$)의 값(tanh -> -1~1 값)과 출력 게이트의 값이 연산되면서, 값이 걸러지는 효과가 발생하여 은닉상태가 됨

### 2. 게이트 순환 유닛 (Gated Recurrent Unit, GRU)
- GRU는 LSTM의 장기 의존성 문제에 대한 해결책을 유지하면서, 은닉 상태를 업데이트 하는 계산을 줄임 (구조 간단히)
- 속도는 빠르지만, 비슷한 성능 (데이터가 적으면 매개 변수의 양이 적은 GRU, 많으면 LSTM)
- (LSTM은 출력, 입력, 삭제 게이트) -> __업데이트 게이트와 리셋 게이트__만 존재
<img src='img/3_13.png' width='400'>